# Allgemein

In [1]:
import z3

# Beispiele

## Brothers and Sisters

I have as many brothers as I have sisters, but my sister has twice as many brothers as she has sisters.
How many children does my father have?

The method `check` examines whether the given set of constraints is satisfiable.
In general, it can return one of the following results:
- `sat`   is returned if the problem is solvable, (`sat` is short for *satisfiable*)
- `unsat` is returned if the problem is unsatisfiable,
- `unknown` is returned if the constraint solver is not powerful enough to solve the given problem.

  In contrast to *ChatGPT*, `Z3` does not start to halucinate if it is unable to solve a given problem, but instead clearly states that it was unable to solve the problem.

In [3]:
boys  = z3.Int('boys')
girls = z3.Int('girls')

S = z3.Solver()

S.add(boys - 1 == girls)
S.add(2 * (girls - 1) == boys)

if str(S.check()) == 'sat':
    print('satisfiable')

solution = S.model()

b = solution[boys ].as_long()
g = solution[girls].as_long()

print(f'My father has {b + g} children: {b} boys and {g} girls.')
print(f'I have {b} - 1 = {b-1} brother and {g} sisters.')
print(f'My sister has {g} - 1 = {g-1} sisters and {b} = 2 * {g-1}  brothers.')

satisfiable
My father has 7 children: 4 boys and 3 girls.
I have 4 - 1 = 3 brother and 3 sisters.
My sister has 3 - 1 = 2 sisters and 4 = 2 * 2  brothers.


## Train

<blockquote id="fancy">

   **Solving a Quadratic Equation with  Z3**

   A train travels at a uniform speed for 360 miles.  
   The train would have taken 48 minutes less to travel the same distance 
   if it had been 5 miles per hour faster. <br/>
    
   **Find the speed of the train**.
</blockquote>

In [7]:
t = z3.Real('t')
v = z3.Real('v')

S = z3.Solver()

S.add(360 == t * v)
S.add(360 == (t - z3.Q(4, 5)) * (v + 5))

S.add(t > 0)

if str(S.check()) == 'sat':
    print('satisfiable')

solution = S.model()

velocity = float(solution[v].as_decimal(17)) # Precision = 17
time = float(solution[t].as_decimal(17))

print(f'velocicty = {velocity}')
print(f'time = {time}')

# Check

print('\nCheck:')
print(time * velocity)
print((time - 4/5) * (velocity + 5))

satisfiable
velocicty = 45.0
time = 8.0

Check:
360.0
360.0


## Parrot and Penguin

- A Japanese deli offers both
  [penguins](https://www.discovermagazine.com/health/hearty-penguin-steaks-the-old-school-explorers-salve-for-scurvy)
     and [parrots](http://fancytoast.blogspot.com/2007/04/parrot-three-ways.html). 
- A parrot and a penguin together cost 666 bucks.
- The penguin costs 600 dollars more than the parrot.
- **What is the price of the parrot?**

In [17]:
parrot = z3.Int('par')
penguin = z3.Int('penguin')

S = z3.Solver()

S.add(parrot + penguin == 666)
S.add(penguin == parrot + 600)

if str(S.check()) == 'sat':
    print('satisfiable')

solution = S.model()

print(f'Parrot = {solution[parrot].as_long()}')

satisfiable
Parrot = 33


## Sudoku

In [8]:
def create_puzzle():
    return [ ["*",  3 ,  9 , "*", "*", "*", "*", "*",  7 ], 
             ["*", "*", "*",  7 , "*", "*",  4 ,  9 ,  2 ],
             ["*", "*", "*", "*",  6 ,  5 , "*",  8 ,  3 ],
             ["*", "*", "*",  6 , "*",  3 ,  2 ,  7 , "*"],
             ["*", "*", "*", "*",  4 , "*",  8 , "*", "*"],
             [ 5 ,  6 , "*", "*", "*", "*", "*", "*", "*"],
             ["*", "*",  5 ,  2 , "*",  9 , "*", "*",  1 ],
             ["*",  2 ,  1 , "*", "*", "*", "*",  4 , "*"],
             [ 7 , "*", "*", "*", "*", "*",  5 , "*", "*"]
           ]

In [9]:
def all_variables():
    Variables = set()
    for row in range(1, 9+1):
        for col in range(1, 9+1):
            Variables.add(f'V{row}{col}')
    return Variables

In [10]:
def allDifferent(L):
    return { f'{x} != {y}' for x in L 
                           for y in L
                           if  x < y
           }

In [11]:
def constraints_from_puzzle():
    Puzzle = create_puzzle()
    return { f'V{row+1}{col+1} == {Puzzle[row][col]}' for row in range(8+1)
                                                      for col in range(8+1)
                                                      if  Puzzle[row][col] != '*'
           }

In [12]:
def all_constraints(): 
    L           = range(1, 9+1)
    Constraints = constraints_from_puzzle()
    # all entries in a row are unique
    for row in L:
        Constraints |= allDifferent({ f'V{row}{col}' for col in L })
    # all entries in a column are unique
    for col in L:
        Constraints |= allDifferent({ f'V{row}{col}' for row in L })
    # all entries in a 3x3 square are unique    
    for r in range(3):
        for c in range(3):
            S = { f'V{r*3+row}{c*3+col}' for row in range(1, 3+1)
                                         for col in range(1, 3+1) 
                }
            Constraints |= allDifferent(S)
    for row in L:
        for col in L:
            Constraints.add(f'0 < V{row}{col}')
            Constraints.add(f'V{row}{col} < 10')
    return Constraints

In [13]:
def solve(Constraints, Variables):
    Environment = {}
    exec('import z3', Environment)
    for v in Variables:
        exec(f'{v} = z3.Int(f"{v}")', Environment)
    s = z3.Solver()
    for c in Constraints:
        s.add(eval(c, Environment))
    result = str(s.check())
    if result == 'sat':
        m = s.model()
        S = { v: m[eval(v, Environment)] for v in Variables }
        return S
    elif result == 'unsat':
        print('The problem is not solvable.')
    else:
        print('Z3 cannot determine whether the problem is solvable.')

In [14]:
%%time
Solution = solve(all_constraints(), all_variables())
Solution

CPU times: total: 203 ms
Wall time: 245 ms


{'V58': 1,
 'V99': 8,
 'V95': 1,
 'V14': 4,
 'V13': 9,
 'V79': 1,
 'V73': 5,
 'V86': 6,
 'V24': 7,
 'V98': 2,
 'V15': 2,
 'V67': 9,
 'V22': 5,
 'V76': 9,
 'V36': 5,
 'V66': 7,
 'V37': 1,
 'V65': 8,
 'V69': 4,
 'V39': 3,
 'V17': 6,
 'V19': 7,
 'V46': 3,
 'V84': 8,
 'V25': 3,
 'V87': 7,
 'V49': 5,
 'V41': 8,
 'V74': 2,
 'V71': 4,
 'V68': 3,
 'V42': 1,
 'V43': 4,
 'V55': 4,
 'V52': 7,
 'V27': 4,
 'V44': 6,
 'V89': 9,
 'V61': 5,
 'V47': 2,
 'V63': 2,
 'V28': 9,
 'V72': 8,
 'V75': 7,
 'V29': 2,
 'V26': 1,
 'V94': 3,
 'V23': 8,
 'V11': 1,
 'V59': 6,
 'V33': 7,
 'V85': 5,
 'V77': 3,
 'V18': 5,
 'V78': 6,
 'V51': 9,
 'V96': 4,
 'V32': 4,
 'V57': 8,
 'V48': 7,
 'V82': 2,
 'V31': 2,
 'V38': 8,
 'V16': 8,
 'V45': 9,
 'V34': 9,
 'V53': 3,
 'V54': 5,
 'V62': 6,
 'V81': 3,
 'V88': 4,
 'V97': 5,
 'V21': 6,
 'V91': 7,
 'V12': 3,
 'V92': 9,
 'V35': 6,
 'V83': 1,
 'V56': 2,
 'V93': 6,
 'V64': 1}